In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read data
df = pd.read_csv('data-sets/air_passengers.csv')

# Create lagged features
for i in range(1, 3):
    df[f'lag_{i}'] = df['passengers'].shift(i)

# Drop NA values
df.dropna(inplace=True)

# Define inputs and target
X = df[['lag_1', 'lag_2']].values
y = df['passengers'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


KeyError: 'passengers'

In [ ]:
import torch
from torch import nn
import pytorch_lightning as pl

class FeedForwardNetwork(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        return self.layer2(x)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

class LightningModel(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

# Preparing data loaders
train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
val_dataset = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Initializing model and training
model = FeedForwardNetwork(input_size=2, hidden_size=100, output_size=1)
lightning_model = LightningModel(model)

trainer = pl.Trainer(max_epochs=100)
trainer.fit(lightning_model, train_loader, val_loader)


In [ ]:
import matplotlib.pyplot as plt

# Predicting
lightning_model.eval()
predictions = lightning_model(torch.Tensor(X_test))

# Plotting
plt.plot(y_test, label='Actual')
plt.plot(predictions.detach().numpy(), label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Values')
plt.xlabel('Time Steps')
plt.ylabel('passengers')
plt.show()
